In [1]:
from struct import pack, unpack
import socket

## Montando o datagram

![structure](assets/snmp_structure.png)  

![data](assets/snmp_data.png)

### Montando VarbindList

In [2]:
snmpValue = pack("2b",5,0) #byte 0x05 indica que dataType do que to enviando é nulo

In [109]:
OID_CODE = '1.2.1.1.5'.split(".")
snmpOid = pack("2b",0x2b, 0x06) + bytearray(int(x) for x in OID_CODE ) # iso.3 + restante do oid

In [110]:
snmpVarbind = snmpOid + snmpValue
snmpVarbind.hex(" ")

'2b 06 01 02 01 01 05 05 00'

In [123]:
snmpVarbindSequence = pack('b',6) + len(snmpOid).to_bytes(1,'little') + snmpVarbind 
snmpVarbindSequence.hex(" ")

'06 07 2b 06 01 02 01 01 05 05 00'

In [126]:
snmpVarbindList = pack('b',0x30) + len(snmpVarbindSequence).to_bytes(1,"little") + snmpVarbindSequence
snmpVarbindList = pack('b',0x30) + len(snmpVarbindList).to_bytes(1,"little") +  snmpVarbindList
snmpVarbindList.hex(" ")

'30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

### Montando PDU

In [144]:
snmpErrorIndex = pack("b",0x0)
snmpPdu = pack("b",0x02) + len(snmpErrorIndex).to_bytes(1,"little") +  snmpErrorIndex + snmpVarbindList
snmpPdu.hex(" ")

'02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

In [145]:
snmpErrorStatus = pack("b",0x0)
snmpPdu = pack('b',2) + len(snmpErrorStatus).to_bytes(1,"little") + snmpErrorStatus + snmpPdu
snmpPdu.hex(" ")

'02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

In [146]:
snmpRequestId = pack("b",0x1)
snmpPdu = pack('b',2) + len(snmpRequestId).to_bytes(1,"little") + snmpRequestId + snmpPdu
snmpPdu.hex(" ")

'02 01 01 02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

In [147]:
snmpPduType =  pack('B',0xa0) # 0xa0 = getRequest, 0xa2 = getResponse
snmpPdu = snmpPduType + len(snmpPdu).to_bytes(1,"little") + snmpPdu
snmpPdu.hex(" ")

'a0 18 02 01 01 02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

### Montando header

In [159]:
COMMUNITY = b'public'
commLen = len(COMMUNITY)
snmpCommunity = pack("{}s".format(commLen),COMMUNITY)

snmpHeader = pack('b',0x04) + commLen.to_bytes(1,"little") + snmpCommunity + snmpPdu

snmpHeader.hex(" ")

'04 06 70 75 62 6c 69 63 a0 18 02 01 01 02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

In [160]:
snmpVersion = pack('b',0x0)
snmpHeader = pack('b',0x2) + len(snmpVersion).to_bytes(1,"little") + snmpVersion + snmpHeader

snmpHeader.hex(" ")

'02 01 00 04 06 70 75 62 6c 69 63 a0 18 02 01 01 02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00'

In [168]:
snmpMessage = pack('b',0x30) + len(snmpHeader).to_bytes(1,"little") + snmpHeader

In [178]:
print('Mensagem SNMP em bytes:\n{}\n\nMensagem SNMP em Hex:\n{}'.format(snmpMessage, snmpMessage.hex(" ")))

Mensagem SNMP em bytes:
b'0%\x02\x01\x00\x04\x06public\xa0\x18\x02\x01\x01\x02\x01\x00\x02\x01\x000\r0\x0b\x06\x07+\x06\x01\x02\x01\x01\x05\x05\x00'

Mensagem SNMP em Hex:
30 25 02 01 00 04 06 70 75 62 6c 69 63 a0 18 02 01 01 02 01 00 02 01 00 30 0d 30 0b 06 07 2b 06 01 02 01 01 05 05 00


## Montando o socket

In [179]:
ipDest = b"127.0.0.1"
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)